In [1]:
import os 
import os.path as osp
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import random
from collections import Counter
import open3d as o3d
import open3d.core as o3c
import json
import subprocess
from PIL import Image
import h5py

import pickle
from  plyfile import PlyData
import torch
from tqdm import tqdm
import torch.multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
#the other imports from the local stuff
import sys

ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import scan3r,visualisation


#reading in the necessary data
data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
frame_number = "000004"
frame_number_2 =  "000016"
img_width = 960
img_height = 540

#"38770c9d-86d7-27b8-869e-4f713b04f290" #is ref 1d2f8510-d757-207c-8c48-3684433860e1
new_scan_id =  "05c6ede7-2e69-23b1-8b27-c1cb868f1938" #"0cac761d-8d6f-2d13-8f35-2364ee20f2a9"#"77361fd2-d054-2a22-88bd-8b14f5969890" #is rescan 9c27de56-6184-2cda-8196-591957b6387d
curr_scan_id = scan3r.get_reference_id(data_dir, new_scan_id)
#the original meshes are given in the file  'labels.instances.annotated.v2.ply'



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/local/home/ekoller/BT


In [2]:
def reconstruct_to_image( patchwise_id):
        patch_width = 30
        patch_height = 30
        
        # Initialize an empty image with zeros (assuming same type as original)
        reconstructed_img = np.zeros((540, 960), dtype=np.int32)
        
        # Loop over patches and place the patchwise_id values into the reconstructed image
        for i in range(18):
            for j in range(32):
                # Define the coordinates of the current patch
                h_start = i * patch_height
                w_start = j * patch_width
                h_end = h_start + patch_height
                w_end = w_start + patch_width
                
                # Assign the patchwise_id value to the corresponding patch area
                reconstructed_img[h_start:h_end, w_start:w_end] = patchwise_id[i, j]
        
        return reconstructed_img

# reference_info_path = osp.join("/local/home/ekoller/R3Scan/files/patch_anno", "patch_anno_{}_{}".format(32,18),"{}.pkl".format(curr_scan_id))
# gt_patches = scan3r.load_pkl_data(reference_info_path)

# gt_patches =gt_patches[frame_number]
# print("gt_patches ")




# img = reconstruct_to_image(gt_patches)
# print(img.shape)

# img_display = cv2.convertScaleAbs(img, alpha=(255.0/np.max(img)))
# cv2.imshow("Reconstructed Image", img_display)
# cv2.waitKey(0)  # Wait indefinitely until a key is pressed
# cv2.destroyAllWindows()


compute the object centers based on gt

In [3]:
#access the data of the object centers
curr_scene_path = osp.join(data_dir, "files", "orig", "data", curr_scan_id + ".pkl")
with open(curr_scene_path, 'rb') as f:
    data = pickle.load(f)

obj_ids_pkl = data["objects_id"]


#access the things for the mesh
pathToMesh = osp.join(data_dir,"scenes", curr_scan_id, "labels.instances.align.annotated.v2.ply")
ply_data = PlyData.read(pathToMesh)
vertices = ply_data['vertex'].data
vertex_array = np.array([list(vertex) for vertex in vertices])

# Extract x, y, z coordinates and objectId
x = vertex_array[:, 0]
y = vertex_array[:, 1]
z = vertex_array[:, 2]
object_ids_mesh = vertex_array[:, 6]  # Assuming 'objectId' is the 7th property

unique_object_ids = np.unique(object_ids_mesh)

bounding_boxes_tmp = {}
centroids = {}
#go over every id and compute the box
for obj_id in unique_object_ids:
                #we want the same ids for the boxes
                if obj_id in obj_ids_pkl:
                        # Filter vertices by object ID
                        obj_mask = object_ids_mesh == obj_id
                        obj_coords = np.vstack((x[obj_mask], y[obj_mask], z[obj_mask])).T
                        
                        #also compute the centroid
                        centroid = np.mean(obj_coords, axis=0)
                        centroids[obj_id] = centroid

print(centroids)

{1.0: array([ 0.03330873,  1.6479874 , -1.2762514 ], dtype=float32), 2.0: array([-0.38284317, -0.6398503 , -0.15691105], dtype=float32), 3.0: array([0.5418692 , 0.03857429, 0.25289795], dtype=float32), 4.0: array([0.52536404, 3.0164602 , 0.4119873 ], dtype=float32), 5.0: array([ 1.0282791 ,  2.6210651 , -0.25832573], dtype=float32), 6.0: array([-0.3130096 ,  3.4233887 , -0.15673104], dtype=float32), 7.0: array([0.23087628, 3.3068411 , 0.10224217], dtype=float32), 8.0: array([1.7231082 , 2.2564652 , 0.46672362], dtype=float32), 9.0: array([ 1.3919603,  1.8465722, -1.1093129], dtype=float32), 10.0: array([ 0.7826736,  2.5110998, -1.1292679], dtype=float32), 11.0: array([ 0.29998198,  2.7308824 , -0.7359261 ], dtype=float32), 12.0: array([-0.1938101,  2.6221223, -0.72542  ], dtype=float32), 14.0: array([-1.8240039,  2.063081 ,  0.2581479], dtype=float32), 15.0: array([-1.7737153 ,  2.0701153 ,  0.39197132], dtype=float32), 16.0: array([-1.7639624 ,  2.0418887 , -0.39463437], dtype=float32

this segment gets for an object id the colour the object is assigned

In [4]:
#for a given scene get the colours of the differnt object_ids as a dictionary
def get_id_colours(data_dir,scan_id):
    #access the mesh file to get the colour of the ids
    mesh_file = osp.join(data_dir,"scenes", scan_id, "labels.instances.annotated.v2.ply")
    ply_data = PlyData.read(mesh_file)
    # Extract vertex data
    vertices = ply_data['vertex']
    vertex_count = len(vertices)
    
    # Initialize dictionary to store object_id -> color mappings
    object_colors = {}
    
   # Iterate through vertices
    for i in range(vertex_count):
        vertex = vertices[i]
        object_id = vertex['objectId']
        color = (vertex['red'], vertex['green'], vertex['blue'])
        
        # Check if object_id already in dictionary, otherwise initialize a Counter
        if object_id in object_colors:
            object_colors[object_id][color] += 1
        else:
            object_colors[object_id] = Counter({color: 1})
    
    # Convert Counter to dictionary with most frequent color
    for object_id, color_counter in object_colors.items():
        most_common_color = color_counter.most_common(1)[0][0]
        object_colors[object_id] = np.array(most_common_color)
    
    return object_colors

In [5]:
def pose_in_reference(data_dir, scan_id , pose_rescan):
    #same coordinate system
    # ref_id = scan3r.get_reference_id(data_dir,scan_id)
    # #if we want the coords in the reference coordinate system return the boxes (based on pkl file)
    # if scan_id==ref_id:
    #     print("we returned the same pose")
    #     return pose_rescan
    

    #transform the centers of rescan to ref coord
    path = osp.join(data_dir,"files", "3RScan.json")
    map_id_to_trans = scan3r.read_transform_mat(path)
    transform = map_id_to_trans[scan_id]
    transform= transform.reshape(4,4)

    #transform the pose

    return    transform.transpose() * pose_rescan

access the depthmap and based on the pose compute the 3d points of this depthmap we do that for the new scan id

In [6]:
import pandas as pd
#frame_number = "000003"
""" access the needed files and stuff like that
"""
#to do: look at the way the pose in reference is done!!!!!

#access the gt projection object ids
gt_obj_ids_path = osp.join(scenes_dir, new_scan_id, "sequence", "frame-{}.color.jpg".format(frame_number))
#access the file
obj_ids = Image.open(gt_obj_ids_path)
#convert to np array
obj_mat = np.array(obj_ids)

#access the extrinsic/pose of the camera
pose_rescan = scan3r.load_pose(osp.join(data_dir, "scenes"), new_scan_id, frame_number)
pose_in_ref = pose_in_reference(data_dir, new_scan_id, pose_rescan)
print("pose in ref shape", pose_in_ref)

#get the intrinsic of the camera
# get img info and camera intrinsics 

#the intrinsics are saved the following way
# intrinsic_mat = np.array([[intrinsic_fx, 0, intrinsic_cx],
#                                     [0, intrinsic_fy, intrinsic_cy],
#                                     [0, 0, 1]])
camera_info = scan3r.load_intrinsics(scenes_dir, new_scan_id)
intrinsics = camera_info['intrinsic_mat']
img_width = int(camera_info['width'])
img_height = int(camera_info['height'])


#access the depht image
depth_path = osp.join(scenes_dir, new_scan_id, "sequence", "frame-{}.depth.pgm".format(frame_number))
#depth_path = osp.join("/media/ekoller/T7/Depth", new_scan_id,"frame-{}.pgm".format(frame_number))

#access the file
pgm_file = Image.open(depth_path)
#convert to np array
#depth_mat_og = np.array(pgm_file)
#print("og map", pgm_file)
#since its distances so discrete things take the nearest value not a different interpolation

obj_mat_size = (obj_mat.shape[1], obj_mat.shape[0]) 
print("obj mat size", obj_mat_size)
depth_mat_resized = pgm_file.resize(obj_mat_size, Image.NEAREST) 
#print("resized map", depth_mat_resized)
#depth is given in mm so put it into m
depth_mat = np.array(depth_mat_resized)
np.set_printoptions(threshold=np.inf)
print("og depthmap", depth_mat)
depth_mat = depth_mat * 0.001
#print("final map", depth_mat)
#print("objects shape", obj_mat.shape)

"""
do the computations based on following formula 

"""

#from 2d to camera coordinates xc = (u-cx)*z / fx,   yc = (v-cy)*z/ fy    , zc= z 


#create a mesh grid since apparently that is how it is done lol
u, v = np.meshgrid(np.arange(img_width), np.arange(img_height))

#also access the intrinsic values
# intrinsic_mat = np.array([[intrinsic_fx, 0, intrinsic_cx],
#                                     [0, intrinsic_fy, intrinsic_cy],
#                                     [0, 0, 1]])

fx = intrinsics[0, 0]  # Focal length in x direction
fy = intrinsics[1, 1]  # Focal length in y direction
cx = intrinsics[0, 2]  # Principal point x
cy = intrinsics[1, 2]  # Principal point y
#flatten everything for computations
u_flat = u.flatten()
v_flat = v.flatten()
depth_flat = depth_mat.flatten()

#apply the formula from above
x_c = (u_flat - cx) * depth_flat / fx
y_c = (v_flat - cy) * depth_flat / fy
z_c = depth_flat

#turn the camera coordinates into homogeneous coordinates
camera_coords_homog  = np.vstack((x_c, y_c, z_c, np.ones_like(x_c)))  

#apply the extrinsic matrix
world_coords_homog =  pose_in_ref @ camera_coords_homog
#normalize
world_coords_homog /= world_coords_homog[3, :]  

world_coords = world_coords_homog[:3,:]
world_coords_T = world_coords.T
print("computed world coords" , world_coords.shape)
print("computed world coords T" , world_coords_T.shape)
#normalize the colour of the gt
rgb_array = np.array(obj_mat) / 255.0
#access the colours 
rgb_colors = rgb_array[v_flat, u_flat]
print("computed colours" , rgb_colors.shape)
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(np.array(world_coords_T))
point_cloud.colors = o3d.utility.Vector3dVector(np.array(rgb_colors))
#o3d.visualization.draw_geometries([point_cloud])

# Set the voxel size (you can adjust this depending on the desired resolution)
voxel_size = 0.08  # Adjust this value based on your needs

# Apply voxel downsampling
downsampled_point_cloud = point_cloud.voxel_down_sample(voxel_size=voxel_size)

# Print the number of points before and after downsampling
#print("Original point cloud size:", len(point_cloud.points))
#print("Downsampled point cloud size:", len(downsampled_point_cloud.points))



# Load the mesh
pathToMesh = osp.join(data_dir, "scenes", new_scan_id, "labels.instances.align.annotated.v2.ply")
new_mesh = o3d.io.read_triangle_mesh(pathToMesh)

# Check if the mesh has colors
if not new_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

# Normalize the mesh colors if necessary
colors = np.asarray(new_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
    colors /= 255.0

# Swap color channels if necessary
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
new_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

""" pointcloud pkl

"""

#also access the ground truth pointclouds
pklfile = osp.join('/local/home/ekoller/R3Scan', 'files', 'orig', 'data', '{}.pkl'.format(new_scan_id))

with open(pklfile, "rb") as f:
    # Load the data from the pickle file
    data = pickle.load(f)
    
    # Extract object points and IDs from the pickle data
    pointclouds = data['obj_points'][512]
    object_ids = data['objects_id']
    
    # Create a new point cloud object for the .pkl data
    pkl_point_cloud = o3d.geometry.PointCloud()
    obj_colors = get_id_colours(data_dir, new_scan_id)
    # Iterate over all the points and their object IDs
    for obj_id, points in zip(object_ids, pointclouds):
        # Assign a default color (e.g., black) to these objects
        #print("object id", obj_id)
        color = obj_colors[obj_id]/225.0
        #print("clour", obj_colors[obj_id]/225.0)
        # Create a point cloud for this object's points
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (points.shape[0], 1)))  # Assign the color

        # Add this point cloud to the combined pkl_point_cloud
        pkl_point_cloud += pcd
# Display both the mesh and point cloud together
o3d.visualization.draw_geometries([point_cloud, new_mesh], window_name="Mesh and Point Cloud Display")



pose in ref shape [[ 0.29104676  0.9125041  -0.28745006  0.12351425]
 [ 0.45843712 -0.39673711 -0.79525842  0.57758014]
 [-0.83971829  0.09967897 -0.53379538  0.07887019]
 [ 0.          0.          0.          1.        ]]
obj mat size (960, 540)
og depthmap [[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0 1133 1133 1133 1133 1133 1133 1133 1133
  1133 1134 1134 1134 1134    0    0    0    0    0    0    0    0 1145
  1145 1145 1145 1145 1148 1148 1148 1148 1148 1148 1148 1148 1148 1148
  1148 1148 1148 1153 1153 1153 1153 1162 1162 1162 1162 1168 1168 1168
  1168 1168 1168 1168

this part now tries to get the depthmaps of the same object in 2 frames and visualize it

In [7]:
#the dino generated mask
#returns featuer in the form of features: frame: list of {objext_id, bbox, mask} objects
def read_segmentation_data(segmentation_path):
    features = {}
    with h5py.File(segmentation_path, 'r') as hdf_file:
            for frame_idx in hdf_file.keys():
                #init boxlist for curr frame
                bounding_boxes = []
                
                # get info 
                frame_group = hdf_file[frame_idx]
                
                #iterate over each boundingbox
                for bbox_key in frame_group.keys():
                    bbox_group = frame_group[bbox_key]
                    
                    #get te obj id
                    object_id = bbox_group.attrs['object_id']
                    
                    #get the boundingbox
                    bbox = bbox_group['bbox'][:]
                    
                    # get the maskt
                    mask = bbox_group['mask'][:]
                    
                    # append to list
                    bounding_boxes.append({
                        'object_id': object_id,
                        'bbox': bbox,
                        'mask': mask
                    })
                
                # stor it to the corresponding frame
                
                features[frame_idx] = bounding_boxes
    return features

  

In [8]:
#transform to reference coordinate system
def transform_to_3d(data_dir, scenes_dir, scan_id, depth_map, colour_map, frame_number):
    """ access the needed files and stuff like that
    """
   
    #access the extrinsic/pose of the camera
    pose_rescan = scan3r.load_pose(osp.join(data_dir, "scenes"), scan_id, frame_number)
    pose_in_ref = pose_in_reference(data_dir, scan_id, pose_rescan)
    
    camera_info = scan3r.load_intrinsics(scenes_dir, scan_id)
    intrinsics = camera_info['intrinsic_mat']
    img_width = int(camera_info['width'])
    img_height = int(camera_info['height'])

    """
    do the computations based on following formula 

    """

    #from 2d to camera coordinates xc = (u-cx)*z / fx,   yc = (v-cy)*z/ fy    , zc= z 


    #create a mesh grid since apparently that is how it is done lol
    u, v = np.meshgrid(np.arange(img_width), np.arange(img_height))

    #also access the intrinsic values
    # intrinsic_mat = np.array([[intrinsic_fx, 0, intrinsic_cx],
    #                                     [0, intrinsic_fy, intrinsic_cy],
    #                                     [0, 0, 1]])

    fx = intrinsics[0, 0]  # Focal length in x direction
    fy = intrinsics[1, 1]  # Focal length in y direction
    cx = intrinsics[0, 2]  # Principal point x
    cy = intrinsics[1, 2]  # Principal point y
    #flatten everything for computations
    u_flat = u.flatten()
    v_flat = v.flatten()
    depth_flat = depth_map.flatten()

    #apply the formula from above
    x_c = (u_flat - cx) * depth_flat / fx
    y_c = (v_flat - cy) * depth_flat / fy
    z_c = depth_flat

    #turn the camera coordinates into homogeneous coordinates
    camera_coords_homog  = np.vstack((x_c, y_c, z_c, np.ones_like(x_c)))  

    #apply the extrinsic matrix
    world_coords_homog = pose_in_ref @ camera_coords_homog
    #normalize
    world_coords_homog /= world_coords_homog[3, :]  

    world_coords = world_coords_homog[:3,:]
    world_coords_T = world_coords.T
    # print("computed world coords" , world_coords.shape)
    # print("computed world coords T" , world_coords_T.shape)
    #normalize the colour of the gt
    rgb_array = np.array(colour_map) / 255.0
    #access the colours 
    rgb_colors = rgb_array[v_flat, u_flat]
   

    return world_coords_T, rgb_colors
    


In [9]:
def voxel_grid_to_coordinates(voxel_grid):
    """Extract voxel coordinates from a VoxelGrid object."""
    voxels = voxel_grid.get_voxels()
    voxel_coords = np.array([voxel.grid_index for voxel in voxels])
    return voxel_coords


def compare_voxel_grids(voxel_grid1, voxel_grid2):
    """Compare two voxel grids to see how much they overlap."""
    coords1 = voxel_grid_to_coordinates(voxel_grid1)
    coords2 = voxel_grid_to_coordinates(voxel_grid2)
    
    # Convert to sets of tuples for intersection
    voxels1_set = set(map(tuple, coords1))
    voxels2_set = set(map(tuple, coords2))
    
    # Compute intersection
    intersection = voxels1_set.intersection(voxels2_set)
    union = voxels1_set.union(voxels2_set)
    
    similarity = len(intersection) / len(union) if len(union) > 0 else 0
    return similarity

# def create_bounding_box_lines(bbox):
#     # Create a LineSet for the bounding box
#     lines = [
#         [0, 1], [1, 2], [2, 3], [3, 0], # bottom face
#         [4, 5], [5, 6], [6, 7], [7, 4], # top face
#         [0, 4], [1, 5], [2, 6], [3, 7]  # vertical lines
#     ]
#     # Convert the bounding box min and max bounds to corner points
#     min_bound = bbox.get_min_bound()
#     max_bound = bbox.get_max_bound()

#     corners = [
#         [min_bound[0], min_bound[1], min_bound[2]],
#         [max_bound[0], min_bound[1], min_bound[2]],
#         [max_bound[0], max_bound[1], min_bound[2]],
#         [min_bound[0], max_bound[1], min_bound[2]],
#         [min_bound[0], min_bound[1], max_bound[2]],
#         [max_bound[0], min_bound[1], max_bound[2]],
#         [max_bound[0], max_bound[1], max_bound[2]],
#         [min_bound[0], max_bound[1], max_bound[2]]
#     ]

#     lines_set = o3d.geometry.LineSet()
#     lines_set.points = o3d.utility.Vector3dVector(corners)
#     lines_set.lines = o3d.utility.Vector2iVector(lines)
#     return 


# def check_mesh_intersection(mesh1, mesh2):
#     # Check if two meshes intersect
#     mesh1.compute_triangle_normals()
#     mesh2.compute_triangle_normals()
#     mesh1_tree = o3d.geometry.KDTreeFlann(mesh1)
#     mesh2_tree = o3d.geometry.KDTreeFlann(mesh2)
    
#     # Check for intersection by sampling points from the first mesh and searching for them in the second mesh
#     for point in mesh1.vertices:
#         [_, idx, _] = mesh2_tree.search_knn_vector_3d(point, 1)
#         if len(idx) > 0:
#             return True
#     return False



In [10]:
#read the matched data


voxel_size_overlap = 0.2
def read_matching_data(scan_id):
        # get the file and iterate through everything to create an object
        matchfile = osp.join("/media/ekoller/T7/Predicted_Matches", scan_id + ".h5")
        with h5py.File(matchfile, 'r') as hdf_file:
            loaded_matches = {}
            
            # Iterate through frame indices
            for frame_idx in hdf_file.keys():
                matches = {}
                
                # Access the group for each frame index
                frame_group = hdf_file[frame_idx]
                
                # Load the frame_id -> obj mappings
                for frame_id in frame_group.keys():
                    obj = frame_group[frame_id][()]
                    matches[frame_id] = int(obj)  # Convert back to int
                
                loaded_matches[frame_idx] = matches 

        return loaded_matches


def isolate_object_coordinates(world_coordinates, mask):
        #make sure it is an array
        mask = np.array(mask)
        #print("points shape", world_coordinates.shape)
        #flatten & turn into boolean mask
        mask = mask.flatten()
        #print("mask shape", mask.shape)
        mask = mask.astype(bool)
        #get the part belonging to the object
        obj_coordinates = world_coordinates[mask]

        return obj_coordinates



def do_pcl_overlap(obj_pcl, cluster):
        #create a voxel grid
        #turn into pointclouds
        obj_point_cloud = o3d.geometry.PointCloud()
        obj_point_cloud.points = o3d.utility.Vector3dVector(obj_pcl)
        voxel_grid1 = o3d.geometry.VoxelGrid.create_from_point_cloud(obj_point_cloud, voxel_size_overlap)

        cluster_point_cloud = o3d.geometry.PointCloud()
        cluster_point_cloud.points = o3d.utility.Vector3dVector(cluster)
        voxel_grid2 = o3d.geometry.VoxelGrid.create_from_point_cloud(cluster_point_cloud, voxel_size_overlap)
    
        
        """Compare two voxel grids to see how much they overlap."""
        return compare_voxel_grids(voxel_grid1, voxel_grid2)


def compare_voxel_grids(voxel_grid1, voxel_grid2):
        """Compare two voxel grids to see how much they overlap."""
        coords1 = voxel_grid_to_coordinates(voxel_grid1)
        coords2 = voxel_grid_to_coordinates(voxel_grid2)
        
        # Convert to sets of tuples for intersection
        voxels1_set = set(map(tuple, coords1))
        voxels2_set = set(map(tuple, coords2))
        
        # Compute intersection
        intersection = voxels1_set.intersection(voxels2_set)
        union = voxels1_set.union(voxels2_set)
        
        similarity = len(intersection) / len(union) if len(union) > 0 else 0
        return similarity

def visualize_cluster_open3d(cluster_points, colors):
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(cluster_points)

    # Set the colors for each point
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    # Visualize using Open3D
    o3d.visualization.draw_geometries([point_cloud])

In [11]:
#new_scan_id = "0cac761d-8d6f-2d13-8f35-2364ee20f2a9"
color_dict = get_id_colours(data_dir, curr_scan_id)
print("color dict", color_dict.keys())
matches = read_matching_data(new_scan_id)
print("matches", matches)
all_clusters ={}
overlap_threshold = 0.5
#new_scan_id = "0cac761d-8d6f-2d13-8f35-2364ee20f2a9" #"fcf66d8a-622d-291c-8429-0e1109c6bb26" #"fcf66d9e-622d-291c-84c2-bb23dfe31327"

#access the dino segmentation
segmentation_info_path = osp.join("/media/ekoller/T7/Segmentation/DinoV2/objects", new_scan_id + ".h5")
segmentation_data = read_segmentation_data(segmentation_info_path)

for frame_idx in matches.keys():
    #print("frame idx", frame_idx)

    #access the matches for this frame
    frame_matches = matches[frame_idx]
    #access the depht image
    depth_path = osp.join(scenes_dir, new_scan_id, "sequence", "frame-{}.depth.pgm".format(frame_idx))
    #access the file
    pgm_file = Image.open(depth_path)

    #since its distances so discrete things take the nearest value not a different interpolation
    depth_mat_resized = pgm_file.resize((img_width, img_height), Image.NEAREST) 

    #depth is given in mm so put it into m
    depth_mat = np.array(depth_mat_resized)
    depth_mat = depth_mat * 0.001
    #print("depth map size", depth_mat.shape)

    #access the colour map
    rgb_path = osp.join(scenes_dir, new_scan_id, "sequence", "frame-{}.color.jpg".format(frame_idx))
    #access the file
    rgb_file = Image.open(rgb_path)

    rgb_mat = np.array(rgb_file)

    #transform to world coordinates in the reference frame
    world_coordinates_frame, colour_coordinates = transform_to_3d(data_dir,scenes_dir,new_scan_id, depth_mat, rgb_mat,frame_idx)
    

    # "viszalize"
    # point_cloud = o3d.geometry.PointCloud()
    # point_cloud.points = o3d.utility.Vector3dVector(np.array(world_coordinates_frame))
    # point_cloud.colors = o3d.utility.Vector3dVector(np.array(colour_coordinates))
    # #o3d.visualization.draw_geometries([point_cloud])

    # # Set the voxel size (you can adjust this depending on the desired resolution)
    # voxel_size = 0.08  # Adjust this value based on your needs

    # # Apply voxel downsampling
    # downsampled_point_cloud = point_cloud.voxel_down_sample(voxel_size=voxel_size)

    # # Print the number of points before and after downsampling
    # #print("Original point cloud size:", len(point_cloud.points))
    # #print("Downsampled point cloud size:", len(downsampled_point_cloud.points))



    # # Load the mesh
    # pathToMesh = osp.join(data_dir, "scenes", new_scan_id, "labels.instances.align.annotated.v2.ply")
    # new_mesh = o3d.io.read_triangle_mesh(pathToMesh)

    # # Check if the mesh has colors
    # if not new_mesh.has_vertex_colors():
    #     print("Mesh does not have vertex colors")
    #     exit()

    # # Normalize the mesh colors if necessary
    # colors = np.asarray(new_mesh.vertex_colors)
    # if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
    #     colors /= 255.0

    # # Swap color channels if necessary
    # colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
    # new_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    # o3d.visualization.draw_geometries([point_cloud, new_mesh], window_name="Mesh and Point Cloud Display")

    "end viszalize"
    #iterate through the masks of the objec
    segmented_img_path = osp.join("/media/ekoller/T7/Segmentation/DinoV2/color", new_scan_id, "frame-{}.jpg".format(frame_idx))
    segmented_img = cv2.imread(segmented_img_path)
               
    for boundingboxes in segmentation_data[frame_idx]:

        mask = boundingboxes['mask']
        mask = mask.astype(bool)
        # print("mask box", mask)
        #print("Mask shape:", mask.shape)
        # print("Mask dtype:", mask.dtype)    
        #access the patches withing the segmented image
        masked_region = segmented_img[mask]
        #determin the most occuring colour
        colors_in_region = list(map(tuple, masked_region.reshape(-1, segmented_img.shape[-1])))
        most_frequent_color = Counter(colors_in_region).most_common(1)[0][0]
        #create a mask of the colour in the whole image
        color_mask = np.all(segmented_img == most_frequent_color, axis=-1)
        # print("color mask ", color_mask)
        #we only want the mask for the region of the first region
        result_mask = color_mask & mask
        #access the mask for the object
        mask = boundingboxes['mask']

        #get the dino object_id 
        dino_id = boundingboxes["object_id"]
        #print("frame ", frame_idx, " dino_id ", dino_id)
        #get the matched id
        object_id = frame_matches[str(dino_id)]
        #print("matched id ", object_id)
        
        

        #isolate only the object pointcloud
        obj_pcl = isolate_object_coordinates(world_coordinates_frame, result_mask)
        obj_colour = isolate_object_coordinates(colour_coordinates, result_mask)
        #not a new object so regular precedure
        if object_id > 0:
            #now we need to find out if we add it to the pointcloud of the object it mapped to or not
            if object_id not in all_clusters:
                #print("create new cluster for objct id ", object_id)
                #print("create first cluter obj_id ", object_id)
                #there are no clusters & votes stored for this object jet
                all_clusters[object_id] = [{'cluster': obj_pcl, 'votes': 1, "colour": obj_colour}]
                
                #visualize_cluster_open3d(all_clusters[object_id][0]["cluster"],all_clusters[object_id][0]["colour"] )
                
            #object already has pointclouds we need to see if we merge or add a new cluster
            else:
                #each new cluster starts unmerged
                merged = False
                max_overlap = 0
                best_cluster_index = None
                for i, cluster_data in enumerate(all_clusters[object_id]):
                    cluster = cluster_data['cluster']

                    #add to the cluster with the most overlap
                    overlap = do_pcl_overlap(obj_pcl, cluster)

                    # keep track of the most overlap cluste
                    if overlap > overlap_threshold and overlap > max_overlap:
                        max_overlap = overlap
                        best_cluster_index = i

                if best_cluster_index is not None:
                    # Merge the point clouds with the best cluster
                    #print("merge cluster for object id", object_id)
                    best_cluster = all_clusters[object_id][best_cluster_index]['cluster']
                    merged_points = np.vstack((obj_pcl, best_cluster))
                    merged_rgb = np.vstack((obj_colour,all_clusters[object_id][best_cluster_index]['colour']))
                    
                    # Update the best cluster with the merged points
                    all_clusters[object_id][best_cluster_index]['cluster'] = merged_points
                    all_clusters[object_id][best_cluster_index]['colour'] = merged_rgb
     
                    #visualize_cluster_open3d(merged_points,merged_rgb)
                    # Increment the vote count for the best cluster
                    all_clusters[object_id][best_cluster_index]['votes'] += 1

                    # Mark as merged
                    merged = True
                if not merged:
                    #print("not merged, create new cluster for obj id", object_id)
                    all_clusters[object_id].append({'cluster': obj_pcl, 'votes': 1, "colour": obj_colour})
        #new object
        else:
            #print("working with new ids")
            #get the negative keys
            negative_keys = [object_id for object_id in all_clusters.keys() if object_id < 0]
            
            #no negative keys yet
            if len(negative_keys) == 0:
                new_obj_idx = new_obj_idx - 1
                all_clusters[new_obj_idx] = [{'cluster': obj_pcl, 'votes': 1}]
            #since we don't know the correspondance of the points we just add id to the new cluster with the most points
            else:
                max_overlap = 0
                best_cluster_index = None
                best_object_id = None

                # iterate over every cluster to get the one with the most overlap
                for neg_key in negative_keys:
                    for i, cluster_data in enumerate(all_clusters[neg_key]):
                        cluster = cluster_data['cluster']
                        overlap = do_pcl_overlap(obj_pcl, cluster)

                        # Track the cluster with the highest overlap
                        if overlap > overlap_threshold and overlap > max_overlap:
                            max_overlap = overlap
                            best_cluster_index = i
                            best_object_id = neg_key

                # we found a best cluster so merge it
                if best_object_id is not None and best_cluster_index is not None:
                    best_cluster = all_clusters[best_object_id][best_cluster_index]['cluster']
                    merged_points = np.vstack((obj_pcl, best_cluster))

                    # Update the best cluster with the merged points
                    all_clusters[best_object_id][best_cluster_index]['cluster'] = merged_points

                    # increment the vote
                    all_clusters[best_object_id][best_cluster_index]['votes'] += 1
                else:
                    # did not find a good cluster create a new one
                    new_obj_idx = new_obj_idx -1
                    all_clusters[new_obj_idx] = [{'cluster': obj_pcl, 'votes': 1}]




# print("clusters keys", all_clusters.keys())
# print("all clusters", all_clusters)
#now that we have the lists of clusters we need to iterate over them and choose the biggest cluster, downsample it & take the average to predict the center
#initialize final object
all_centers = {}
#iterte through the objects
for obj_id, clusters in all_clusters.items():
    #print("in the for loop with cluster id", obj_id)
    #get the cluster with the most points aka largest 
    #print("clusters", clusters , "for object id " ,obj_id)
    #decide the most likely correct cluster based on votes first and then size
    largest_cluster_data = max(all_clusters[obj_id], key=lambda c: (c['votes'], len(c['cluster'])))
    largest_cluster = largest_cluster_data['cluster']
    largest_cluster_votes = largest_cluster_data["votes"]
    #print("shape of largest cluster", largest_cluster.shape)
    # Compute the mean (centroid) of the inliers
    obj_center = np.mean(largest_cluster, axis=0)
  


    

    #return the object for the evaluation
    all_centers[obj_id] = {
        'center': obj_center,
        "size": len(largest_cluster),
        "votes" : largest_cluster_votes,
        "points": largest_cluster

    }
    #print(all_centers)

color dict dict_keys([15, 32, 24, 14, 3, 17, 9, 28, 19, 16, 18, 100, 1, 4, 0, 6, 7, 2, 22, 30, 5, 11, 12, 34, 10, 8, 23, 26, 27, 21, 20, 29, 25])
matches {'000000': {'1': 2, '2': 3}, '000001': {'1': 32, '2': 32}, '000002': {'1': 2, '2': 32}, '000003': {'1': 2, '2': 2}, '000004': {'1': 32, '2': 1, '3': 24, '4': 11, '5': 3, '6': 11, '7': 4}, '000005': {'1': 32, '2': 1, '3': 4, '4': 3, '5': 11}, '000006': {'1': 2, '2': 1, '3': 4, '4': 8, '5': 22}, '000007': {'1': 28, '10': 22, '2': 2, '3': 1, '4': 22, '5': 4, '6': 24, '7': 19, '8': 24, '9': 8}, '000008': {'1': 20, '10': 22, '2': 8, '3': 34, '4': 1, '5': 14, '6': 30, '7': 19, '8': 8, '9': 30}, '000009': {'1': 28, '10': 18, '11': 22, '2': 5, '3': 30, '4': 1, '5': 17, '6': 1, '7': 19, '8': 8, '9': 22}, '000010': {'1': 24, '10': 11, '11': 1, '2': 30, '3': 23, '4': 11, '5': 17, '6': 19, '7': 21, '8': 22, '9': 18}, '000011': {'1': 25, '2': 20, '3': 25, '4': 17, '5': 20, '6': 8, '7': 18, '8': 21}, '000012': {'1': 24, '2': 17, '3': 14, '4': 8, '5

In [14]:

""" 
"to doooooooo!!!!!!"
so basically we saw that the votes are not the most imoortant thing but rather the point sizes so adjust the computation for the following things: votes
threshold for the size of downsampling; 0.05,0.1
number of points[10,20,30,40,50,60]
threshold for the overlap of the region [0.1,0.3,0.5,0.7,0.9]
threshold for k nearest neigbour: [1,2,3,4,5,,6,7,8,9]


"""

print("all centers", len(all_centers))
"to doooooooo!!!!!!"
 #access gt pointcenters
pklfile = osp.join(data_dir, 'files', 'orig', 'data', '{}.pkl'.format(new_scan_id))

with open(pklfile, "rb") as f:
    # Load the data from the pickle file
    data = pickle.load(f)
    
# extract object points and IDs from the pickle data
gt_ids = data['objects_id']
gt_centers = data["object_centers"]
gt_boxes = data['bounding_boxes']



"""
New_scan_id mesh display in original coordinate system (rescan coord)
"""
pathToMesh = osp.join(data_dir,"scenes", new_scan_id, "labels.instances.align.annotated.v2.ply")
new_mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#check if it has colours
if not new_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(new_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(new_mesh.vertex_colors)
#colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
new_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)




vis = o3d.visualization.Visualizer()
vis.create_window(width=800, height=600)

#alsoiterate through the newly predicted centers
color_dict = get_id_colours(data_dir, curr_scan_id)
print("predicted centers", all_centers.keys)
for obj in all_centers.keys():
    if obj in gt_centers:
        vis = o3d.visualization.Visualizer()
        vis.create_window(width=800, height=600)
        #vis.add_geometry(new_mesh)
        #print("votes",all_centers[obj]["votes"])
    
        pcl = all_centers[obj]["points"]
        #print("number of points", len(pcl))
        color = color_dict[obj]
        color = color/255.0
        center_colors = np.array([color for _ in range(pcl.shape[0])]) 
        center_point_cloud = o3d.geometry.PointCloud()
        center_point_cloud.points = o3d.utility.Vector3dVector(pcl)
        center_point_cloud.colors = o3d.utility.Vector3dVector(center_colors)  # Set the color for center points
        voxel_size = 0.075  # Adjust this value based on your needs
        downsampled_point_cloud = center_point_cloud.voxel_down_sample(voxel_size=voxel_size)
        cl, ind = downsampled_point_cloud.remove_statistical_outlier(nb_neighbors=20, std_ratio=1.75)
        pcl = np.asarray(cl.points)

        print("number of points", pcl.shape)
        vis.add_geometry(cl)

        #vis.add_geometry(center_point_cloud)

        #center = all_centers[obj]["center"]
        center= np.median(pcl, axis= 0)
        #print("center shape", center.shape)
        color = color_dict[obj]
        color = color/255.0

        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)  # Adjust radius for size
        sphere.paint_uniform_color(color)  # Set color for the sphere
        sphere.translate(center.T)  # Move sphere to the point location
        vis.add_geometry(sphere)  # Add sphere to the visualizer
        #vis.run()
        #vis.destroy_window()
        # Apply voxel downsampling
        #downsampled_point_cloud = center_point_cloud.voxel_down_sample(voxel_size=voxel_size)
        #vis.add_geometry(downsampled_point_cloud)

        #also add the boundingbox
        corners = gt_boxes[obj]
        lines = [
            [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
            [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
            [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical lines
        ]
        
        # Create LineSet for the bounding box
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(corners)
        line_set.lines = o3d.utility.Vector2iVector(lines)
        line_set.paint_uniform_color([0, 0, 0])  # Color the bounding box red
        vis.add_geometry(line_set)

        min_coords = np.min(corners, axis=0)
        max_coords = np.max(corners, axis=0)

        print("center", center)
        print("min coordinates", min_coords)
        print("max coords", max_coords)
        is_inside = (np.all(min_coords <= center) and np.all(center <= max_coords))

        # if is_inside:
        #     print("the object was inside")
        print("is inside" ,is_inside)
        vis.run()
        vis.destroy_window()





"""
Display everything
"""

# render_option = vis.get_render_option()
# render_option.point_size = 10.0  # Increase this value to enlarge the points

#vis.add_geometry(new_mesh)
#vis.run()
#vis.destroy_window()

all centers 32
predicted centers <built-in method keys of dict object at 0x7fb715df26c0>
number of points (489, 3)
center [ 1.32346604  1.46052535 -0.18021198]
min coordinates [-0.84868073 -0.8881131  -1.3059072 ]
max coords [ 0.02003298 -0.33982342  0.781799  ]
is inside False
number of points (104, 3)
center [ 1.07737986  0.98971499 -0.05797239]
min coordinates [-0.01007574 -0.89650106 -1.3204603 ]
max coords [1.8177738 2.2975097 1.403076 ]
is inside True


KeyboardInterrupt: 

In [44]:
#access the matched ids
matches = read_matching_data(new_scan_id)


#access the dino segmentation
segmentation_info_path = osp.join("/media/ekoller/T7/Segmentation/DinoV2/objects", new_scan_id + ".h5")
segmentation_data = read_segmentation_data(segmentation_info_path)

frame_boxes = segmentation_data[frame_number][2]
print(frame_boxes)
mask = frame_boxes["mask"]



#access the rgb of the image
rgb_path = osp.join(scenes_dir, new_scan_id, "sequence", "frame-{}.color.jpg".format(frame_number))
#access the file
rgb_file = Image.open(rgb_path)

rgb_mat = np.array(rgb_file)
print("rgb shape", rgb_mat.shape)


#access the deph map of a frame
depth_path = osp.join(scenes_dir, new_scan_id, "sequence", "frame-{}.depth.pgm".format(frame_number))
#access the file
pgm_file = Image.open(depth_path)

#since its distances so discrete things take the nearest value not a different interpolation
depth_mat_resized = pgm_file.resize((img_width, img_height), Image.NEAREST) 

#depth is given in mm so put it into m
depth_mat = np.array(depth_mat_resized)
depth_mat = depth_mat * 0.001


#create the 3d projection
world_coord, rgb_coord = transform_to_3d(data_dir, scenes_dir, new_scan_id, depth_mat, rgb_mat, frame_number)


og_point_cloud = o3d.geometry.PointCloud()
og_point_cloud.points = o3d.utility.Vector3dVector(np.array(world_coord))
og_point_cloud.colors = o3d.utility.Vector3dVector(np.array(rgb_coord))
 

"""Do the same thing for the second frame

"""

#access the segmentation of the scan_id

frame_boxes = segmentation_data[frame_number_2][2]
print(frame_boxes)
mask_2 = frame_boxes["mask"]


#access the rgb of the image
rgb_path_2 = osp.join(scenes_dir, new_scan_id, "sequence", "frame-{}.color.jpg".format(frame_number_2))
#access the file
rgb_file_2 = Image.open(rgb_path_2)

rgb_mat_2 = np.array(rgb_file_2)


#access the deph map of a frame
depth_path = osp.join(scenes_dir, new_scan_id, "sequence", "frame-{}.depth.pgm".format(frame_number_2))
#access the file
pgm_file = Image.open(depth_path)

#since its distances so discrete things take the nearest value not a different interpolation
depth_mat_resized_2 = pgm_file.resize((img_width, img_height), Image.NEAREST) 

#depth is given in mm so put it into m
depth_mat_2 = np.array(depth_mat_resized_2)
depth_mat_2 = depth_mat * 0.001



world_coord_2, rgb_coord_2 = transform_to_3d(data_dir, scenes_dir, new_scan_id, depth_mat_2, rgb_mat_2, frame_number_2)



""" based on the points filter the ones out which correspond to our points, turn this into point clouds
"""
mask = np.array(mask)  # Replace with your mask array

# Convert mask to an image
mask_image = Image.fromarray(mask.astype(np.uint8) * 255)  # Convert mask to 8-bit grayscale

mask_image.show()
mask = mask.flatten()
mask =  mask.astype(bool)
print(mask)
obj_points = world_coord[mask]
print("shape obje point", obj_points.shape)
obj_rgb = rgb_coord[mask]
#turn into a pointcloud
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(np.array(obj_points))
point_cloud.colors = o3d.utility.Vector3dVector(np.array(obj_rgb))
 




mask_2 = np.array(mask_2)  # Replace with your mask array

# Convert mask to an image
mask_image_2 = Image.fromarray(mask_2.astype(np.uint8) * 255)  # Convert mask to 8-bit grayscale

mask_image_2.show()

mask_2 = mask_2.flatten()
mask_2 = mask_2.astype(bool)
obj_points_2 = world_coord_2[mask_2]
print("obje points shape", obj_points.shape)
obj_rgb_2 = rgb_coord_2[mask_2]
#turn into a pointcloud
point_cloud_2 = o3d.geometry.PointCloud()
point_cloud_2.points = o3d.utility.Vector3dVector(np.array(obj_points_2))
point_cloud_2.colors = o3d.utility.Vector3dVector(np.array(obj_rgb_2))



"""look at voxelgridoverlap
"""
voxel_size = 0.05 # Adjust voxel size as needed
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(point_cloud, voxel_size)
voxel_grid2 = o3d.geometry.VoxelGrid.create_from_point_cloud(point_cloud_2, voxel_size)

similarity = compare_voxel_grids(voxel_grid, voxel_grid2)

if similarity:
    print("The new points likely represent the same object. Similarity is", similarity)

    # Combine the points if they represent the same object
    all_pointcloud = o3d.geometry.PointCloud()
    all_points = np.vstack((np.asarray(point_cloud.points), np.asarray(point_cloud_2.points)))
    all_colors = np.vstack((np.asarray(point_cloud.colors), np.asarray(point_cloud_2.colors)))

    # Update the point cloud with the combined points and colors
    all_pointcloud.points = o3d.utility.Vector3dVector(all_points)
    all_pointcloud.colors = o3d.utility.Vector3dVector(all_colors)

    # Visualize both point clouds and their bounding boxes
    o3d.visualization.draw_geometries([all_pointcloud])
else:
    print("The new points likely represent a different object. Similarity is ", similarity)

    # Visualize both point clouds and their bounding boxes
    o3d.visualization.draw_geometries([point_cloud, point_cloud_2, og_point_cloud])


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/media/ekoller/T7/Predicted_Matches/fcf66d8a-622d-291c-8429-0e1109c6bb26.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)